### Experiment with using Hooke latent space for embryo projections

In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import os
import glob2 as glob
import patsy

# set paths
fig_root = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/figures/seq_data/PLN/"

# specify which regression to use
ccm = "t_spline_inter3" #"t_spline_inter"

fig_folder = os.path.join(fig_root, ccm, "")
os.makedirs(fig_folder, exist_ok=True)

# set path to data
hooke_data_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/seq_data/emb_projections/hooke_model_files/"
ccs_data_path = "/Users/nick/Cole Trapnell's Lab Dropbox/Nick Lammers/Nick/morphseq/seq_data/emb_projections/ccs_data_cell_type_broad/"
model_path = os.path.join(hooke_data_path, ccm, "")

### Load in metadata, model params, and counts matrices 

In [2]:
# load full counts dataset
hooke_counts_long = pd.read_csv(model_path + "abundance_estimates.csv", index_col=0)
cols = list(hooke_counts_long.columns)
cell_ind = cols.index("cell_group")
cov_cols = cols[:cell_ind]
hooke_counts_df = hooke_counts_long.pivot(index=cov_cols,
                                           columns=["cell_group"], values = ["log_abund"])
hooke_counts_df.columns = ['_'.join(map(str, col)).strip('_') for col in hooke_counts_df.columns.values]
hooke_counts_df.reset_index(inplace=True)
new_cols = [col.replace("log_abund_", "") for col in hooke_counts_df.columns.values]
hooke_counts_df.columns = new_cols
sort_cols = new_cols[:cell_ind] + sorted(new_cols[cell_ind:], key=str.lower)
hooke_counts_df = hooke_counts_df.loc[:, sort_cols]

# make stripped-down metadata df
meta_df = hooke_counts_df[cov_cols].copy()
meta_df.loc[:, "dummy_response"] = 0

# model formula
with open(model_path + "model_string.txt", "r") as file:
    formula_str = file.read()
formula_str = "dummy_response " + formula_str 
formula_str = formula_str.replace("ns(", "cr(")
formula_str = formula_str.replace("c(", "(")
formula_str = formula_str.replace("\n", "")

# load hooke predictions (for comparison purposes)
latent_df = pd.read_csv(model_path + "latents.csv", index_col=0)
time_splines = pd.read_csv(model_path + "time_splines.csv")

# load hooke model files
# b_array = pd.read_csv(model_path + "B.csv", index_col=0)
cov_array = pd.read_csv(model_path + "COV.csv", index_col=0)
beta_array = pd.read_csv(model_path + "B.csv", index_col=0).T

# latent_df.head()
beta_array = beta_array.rename(columns={"(Intercept)":"Intercept"})
cols_from = beta_array.columns
cols_from_clean = [col.replace(" = c", "=") for col in cols_from]
beta_array.columns = cols_from_clean
beta_array.head()

,Intercept,"ns(timepoint, knots=(28.5, 51, 73.5))1","ns(timepoint, knots=(28.5, 51, 73.5))2","ns(timepoint, knots=(28.5, 51, 73.5))3","ns(timepoint, knots=(28.5, 51, 73.5))4",dis_protocol,exptCHEM2,exptCHEM3,exptCHEM5,exptGAP13,exptGAP14,exptGAP16,exptGAP18,exptHF4,expthotfish2,exptREF1,"ns(timepoint, knots=(28.5, 51, 73.5))1:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))2:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))3:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))4:dis_protocol"
adenohypophysis,-0.496078,0.841948,1.107938,-1.555601,1.630829,-1.098184,0.136667,-0.485007,-1.407277,-1.365259,-0.578510,-0.258444,-0.148815,-0.794681,-1.147725,-0.434220,0.240736,0.482198,3.557939,-0.378103
adrenal gland,-1.814042,2.015353,4.894048,-0.575699,4.161103,0.007751,-0.250772,-1.042806,-3.849458,-0.262136,0.052147,0.027810,0.898090,-1.480764,0.320295,1.055328,-2.087997,-4.970865,-2.298769,-4.155990
annular ligament (doublets?),1.153327,2.114316,3.745463,-1.731080,7.151964,-1.359858,0.144598,-0.528973,-3.951906,-5.273558,-5.568966,0.118168,1.636440,-3.787234,-1.020568,-8.895006,-0.273235,-1.655863,-5.012906,-0.504264
anterior eye,-2.964543,2.812734,3.169770,7.170701,3.787265,1.352020,-0.224479,-0.540434,-0.942577,-1.232248,-0.745440,-0.625006,0.213320,-0.026404,-1.391667,-2.149864,0.230724,-1.347048,-1.758573,-1.799894
apical ectodermal ridge,-3.327873,7.792945,6.046793,14.458558,5.435542,1.195726,-0.095148,-0.481309,-0.110157,-1.238860,-0.331090,-0.493474,-0.168954,-0.285752,0.564575,-0.370616,-1.678648,-2.208385,-6.793787,-1.370673


### Define helper functions for spline interpolation and covariate matrix generation

In [3]:
from scipy.interpolate import interp1d
# Assume the lookup table has columns: "timepoint", "V1", "V2", "V3", "V4"
# (The actual names might differ; adjust as necessary.)

# Define a function to interpolate the spline basis for a new time value.
def get_spline_basis(new_time_vec, lookup_df):
    # Create an empty dictionary to hold the interpolated values.
    out_df = pd.DataFrame(new_time_vec, columns=["timepoint"])
    
    # Loop through each spline column (skip the "timepoint" column).
    for col in lookup_df.columns[1:]:
        # Create an interpolation function for this column.
        f_interp = interp1d(lookup_df["timepoint"], lookup_df[col],
                            kind='linear', fill_value="extrapolate")
        # Evaluate the interpolation at the new time value.
        out_df[col] = f_interp(new_time_vec)
    return out_df

In [4]:
def get_covariate_df(formula_str, meta_df, time_splines):
    meta_df["dummy_response"] = 0
    _, X = patsy.dmatrices(formula_str, meta_df, return_type='dataframe')
    col_list = list(X.columns)
    cols_to_clean = [col.replace("[T.", "") for col in col_list]
    cols_to_clean = [col.replace("]", "") for col in cols_to_clean]
    cols_to_clean = [col.replace("[", "") for col in cols_to_clean]
    cols_to_clean = [col.replace("cr", "ns") for col in cols_to_clean]
    cols_to_keep = [col for col in cols_to_clean if col in cols_from_clean]
    X.columns = cols_to_clean
    X = X.loc[:, cols_to_keep]
    
    # replace spline cols with lookups (can't get patsy to match ns from R)
    spline_cols = [col for col in cols_to_keep if "ns(" in col]
    spline_vals = get_spline_basis(meta_df.loc[:, "timepoint"].to_numpy(), time_splines)
    if "inter" in ccm:
        X.loc[:, spline_cols[:4]] = spline_vals.iloc[:, 1:].to_numpy()
        X.loc[:, spline_cols[4:]] = np.multiply(spline_vals.iloc[:, 1:].to_numpy(), X.loc[:, "dis_protocol"].to_numpy()[:, None])
    else:
        X.loc[:, spline_cols] = spline_vals.iloc[:, 1:].to_numpy()

    return X, spline_vals.iloc[:, 0].to_numpy()

### Generate WT reference trajectory
Assume bead dissociation and remove experiment-specific effects

In [5]:
import itertools

# set parameters
dis_val = 2 # bead method
expt_val = 0 # remove experiment-specific offsets

nt = 100 # number of timepoints to calculate latent predictions
dis_protocol_vals = np.unique(meta_df["dis_protocol"]).tolist()
expt_vals = np.unique(meta_df["expt"]).tolist()
time_vals = np.linspace(12, 74, nt).tolist()

query_df = pd.DataFrame(itertools.product(time_vals, dis_protocol_vals, expt_vals), columns=["timepoint", "dis_protocol", "expt"])

# get covariate matrix
X_ref, t_vec = get_covariate_df(formula_str, query_df, time_splines)
expt_cols = [col for col in X_ref.columns if "expt" in col]
ref_filter = (X_ref.loc[:, "dis_protocol"]==dis_val) & (np.all(X_ref.loc[:, expt_cols].to_numpy()==0, axis=1))
# null_expt_filter = np.all(X_ref.loc[:, expt_cols].to_numpy()==0, axis=1)
# null_expt_filter = X_ref.loc[:, "dis_protocol"]==2

# expt_array = np.c_[np.zeros((X_ref.shape[0], 1)), X_ref.loc[:, expt_cols].to_numpy()]
# expt_vec = np.argmax(expt_array, axis=1)
# expt_names = ["baseline (bead)"] + [col.replace("expt", "") for col in expt_cols]

X_ref = X_ref.loc[ref_filter, :].reset_index(drop=True)
t_vec = t_vec[ref_filter]

# get log abundance predictions
wt_ref_pd = np.matmul(X_ref, beta_array.T)
wt_ref_pd.head()

/var/folders/m7/tpjttxb92tl8b4c9svgvgby00000gn/T/ipykernel_7472/2638336622.py:29: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  wt_ref_pd = np.matmul(X_ref, beta_array.T)


,0,1,2,3,4,5,6,7,8,9,...,179,180,181,182,183,184,185,186,187,188
0,0.085482,0.731844,-18.994948,-7.143790,-6.238512,-6.282337,2.857196,-0.747556,20.254584,-11.531154,...,-2.291618,-0.012664,-2.877315,-5.716361,-1.600837,0.152016,0.846123,0.569016,4.901258,18.115792
1,0.228464,0.771451,-19.227544,-7.132053,-6.455195,-6.520989,3.015150,-0.720010,20.765695,-11.802679,...,-2.327575,-0.121837,-3.027246,-5.771432,-1.756201,-0.105748,0.846022,0.686845,5.055138,18.591479
2,0.368957,0.809326,-19.456106,-7.120565,-6.668976,-6.756674,3.171003,-0.692593,21.269216,-12.071310,...,-2.363340,-0.228599,-3.174813,-5.825657,-1.907870,-0.360060,0.846762,0.803506,5.207769,19.060776
3,0.506725,0.845305,-19.680254,-7.109350,-6.879580,-6.989111,3.324555,-0.665316,21.764431,-12.336775,...,-2.398893,-0.332722,-3.319793,-5.878959,-2.055493,-0.610595,0.848420,0.918890,5.359032,19.523079
4,0.641533,0.879224,-19.899607,-7.098431,-7.086733,-7.218021,3.475608,-0.638191,22.250621,-12.598799,...,-2.434217,-0.433978,-3.461963,-5.931258,-2.198722,-0.857026,0.851079,1.032887,5.508811,19.977785


### Step 1: can we assign accurate pseudotime values to reference WT embryos?

In [6]:
# # stripped-down function to generate covariate table
# cov_col_list = ['Intercept', 'exptCHEM2', 'exptCHEM3', 'exptCHEM5', 'exptGAP13',
#        'exptGAP14', 'exptGAP16', 'exptGAP18', 'exptHF4', 'exptREF1',
#        'exptREF2', 'expthotfish2',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))1',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))2',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))3',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))4', 'dis_protocol',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))1:dis_protocol',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))2:dis_protocol',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))3:dis_protocol',
#        'ns(timepoint, knots=(26.25, 49.5, 72.75))4:dis_protocol']

cov_col_list = beta_array.columns.tolist()

spline_cols = [col for col in cov_col_list if "ns(" in col]

def construct_X(timepoint, dis, expt, t_splines=time_splines, spline_cols=spline_cols):
    X = pd.DataFrame(np.zeros((1, len(cov_col_list))), columns=cov_col_list)
    X["Intercept"] = 1.0
    X["dis_protocol"] = dis
    expt_i = [i for i in range(len(cov_col_list)) if expt in cov_col_list[i]][0]
    X[cov_col_list[expt_i]] = 1.0
    
    spline_vals = get_spline_basis(np.asarray([timepoint]), time_splines)
    
    X.loc[:, spline_cols[:4]] = spline_vals.iloc[:, 1:].to_numpy()
    X.loc[:, spline_cols[4:]] = np.multiply(spline_vals.iloc[:, 1:].to_numpy(), X.loc[:, "dis_protocol"].to_numpy()[:, None])

    return X

def update_X(X, timepoint, spline_cols=spline_cols):
    spline_vals = get_spline_basis(np.asarray([timepoint]), time_splines)
    
    X.loc[:, spline_cols[:4]] = spline_vals.iloc[:, 1:].to_numpy()
    X.loc[:, spline_cols[4:]] = np.multiply(spline_vals.iloc[:, 1:].to_numpy(), X.loc[:, "dis_protocol"].to_numpy()[:, None])

    return X

In [7]:
from scipy.special import factorial
from scipy.stats import poisson
from scipy.optimize import minimize

# define key helper functions
def calc_zi(log_lambda, theta_array, X):
    Zi = log_lambda - np.matmul(X, theta_array.T)
    return Zi.to_numpy()[0]

def calc_logL_gauss(PHI, Zi): # note that the leaves out the normalization factor. If this is slow, consider simplifying for diagonal Cov
    logL = -0.5 * (Zi[None, :] @ PHI @ Zi[:, None])
    return logL[0][0]

def calc_logL_poiss(raw_counts, log_lambda):
    # logL = np.sum(np.multiply(raw_counts, log_lambda) - np.exp(log_lambda) - factorial(raw_counts))
    log_pmf = poisson.logpmf(raw_counts, np.exp(log_lambda))
    return np.sum(log_pmf)


### Load in REF1 counts

In [8]:
ccs_raw = pd.read_csv(os.path.join(ccs_data_path, "REF1_counts_table.csv"), index_col=0).T
ccs_meta = pd.read_csv(os.path.join(ccs_data_path, "REF1_metadata.csv"), index_col=0)

# augment ccs table to incorporate missing cell types
mdl_cell_types = wt_ref_pd.columns
ccs = ccs_raw.reindex(columns=mdl_cell_types, fill_value=0)

####
# experiment parameters
####
expt = "REF1"
dis = 2 # bead
t_sigma = 10

####
# model parameters
####
# inverse cov matrix
PHI = np.linalg.inv(cov_array)
COV = cov_array.to_numpy()
# covariates
X0 = construct_X(20, dis, expt) 
# regression vars
THETA = beta_array.copy()

In [9]:
X0.head()

,Intercept,"ns(timepoint, knots=(28.5, 51, 73.5))1","ns(timepoint, knots=(28.5, 51, 73.5))2","ns(timepoint, knots=(28.5, 51, 73.5))3","ns(timepoint, knots=(28.5, 51, 73.5))4",dis_protocol,exptCHEM2,exptCHEM3,exptCHEM5,exptGAP13,exptGAP14,exptGAP16,exptGAP18,exptHF4,expthotfish2,exptREF1,"ns(timepoint, knots=(28.5, 51, 73.5))1:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))2:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))3:dis_protocol","ns(timepoint, knots=(28.5, 51, 73.5))4:dis_protocol"
0,1.0,0.040151,-0.137402,0.412205,-0.274804,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.080302,-0.274804,0.824411,-0.549607


In [10]:
def calculate_logL(params, raw_counts, X0=X0, THETA=THETA, PHI=PHI, COV=COV):
    # extract params
    t = params[0]
    Zi = params[1:]
    
    # generate updated covariate matrix
    X = update_X(X0, t)
    mu = (X @ THETA.T).to_numpy().ravel()
    # L, Zi = calculate_L(X, THETA, COV, raw_counts)
    
    # calculate Zi and gaussian logL
    # Zi = (L - mu)
    logL_g = calc_logL_gauss(PHI, Zi)

    # caculate Poisson logL 
    L = Zi + mu
    logL_p = calc_logL_poiss(raw_counts, L)

    return -(logL_g + logL_p)/len(raw_counts)

In [11]:
# embryo_id = embryo_id_vec[-2]
# print(ccs_meta.loc[embryo_id, "timepoint"])
# raw_counts = ccs.loc[embryo_id, :].to_numpy()

# p0 = [36] + [0] * len(raw_counts)

# result = minimize(calculate_logL_v2, p0 , args=(raw_counts,))
# result

In [12]:
from tqdm import tqdm 
zi0_vec = [0] * COV.shape[0]

t_vec = np.linspace(12, 72)
n_test = 15
np.random.seed(541)
embryos_to_test = np.random.choice(np.where((ccs_meta.loc[:, "timepoint"] >=12) & (ccs_meta.loc[:, "timepoint"] <=72))[0], n_test)
embryo_id_vec = np.asarray(ccs.index)[embryos_to_test]

# logL_array = np.empty((len(t_vec), len(embryos_to_test)))
expt_time_vec = np.empty((len(embryos_to_test),))
opt_t_vec = np.empty((len(embryos_to_test),)) 
se_t_vec = np.empty((len(embryos_to_test),)) 
opt_Z_array = np.empty((len(zi0_vec), len(embryos_to_test))) 
se_Z_array = np.empty((len(zi0_vec), len(embryos_to_test))) 

for e, embryo_id in enumerate(tqdm(embryo_id_vec)):

    # embryo_ind = 149
    # get raw counts
    # embryo_id = list(ccs.index)[embryo_ind]
    raw_counts = ccs.loc[embryo_id, :].to_numpy()
    expt_time_vec[e] = ccs_meta.loc[embryo_id, "timepoint"]

    # randomly choose t0 
    dt = np.random.normal(loc=0, scale=t_sigma)
    t0 = np.min([np.max([expt_time_vec[e] + dt, 12]), 72])
    result = minimize(calculate_logL, [t0] + zi0_vec, args=(raw_counts,))
    standard_errors = np.sqrt(np.diag(result.hess_inv))
    
    opt_t_vec[e] = result.x[0]
    se_t_vec[e] = standard_errors[0]

    opt_Z_array[:, e] = result.x[1:]
    se_Z_array[:, e] = standard_errors[1:]
    
    # for i, t in enumerate(t_vec):
    #      logL_array[i, e] = calculate_logL([t], raw_counts)

  0%|          | 0/15 [00:29<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
fig = px.scatter(x=expt_time_vec, y=opt_t_vec, error_y=se_t_vec, trendline="ols")

fig.update_layout(xaxis=dict(range=[0, 72]), 
                  yaxis=dict(range=[0, 72]))
fig.update_layout(width=800, height=600) 
fig.show()

results = px.get_trendline_results(fig)
print(results)

In [ ]:
results.iloc[0,0].params

In [ ]:
se_t_vec[9]

In [ ]:
# run it 

# initial guessess
initial_params = [40]

# Minimize the negative log-likelihood
result = minimize(calculate_logL, initial_params, args=(raw_counts,))

mu = result.x[0]
ste = np.sqrt(result.hess_inv[0, 0])
print(mu)
print(ste)

In [ ]:
calculate_logL(initial_params, raw_counts)

In [ ]:
np.sqrt(result.hess_inv[0, 0])